In [47]:
#Imports
import requests as rq
import json
import requests
import urllib2
from bs4 import BeautifulSoup
import re
import string
from afinn import Afinn
import os
import matplotlib.pyplot as plt


In [48]:
#rapgenius information
rap_genius_id = "_17NJOxR9_RGp1XTQ1Vl3O--bxYh-cSzcCwASKlr-Xlyj3vh2_BsdedBZLX6xgdt"
client_secret = "2wlydcjFiqOlSI5Zi65OeizvaWRx16xkkjIBfPkqZ9t1aFNNYTrjza8ny6xYlXdJQSP_vMzj6e20O7ORtjIIcQ"
client_access_token = "Uzv_tj6uOXE9_1eX2wCrOGi3j4JtmYcm1kVF-UmPcC5gJh1MQZKJLph4aFp9frFx"

In [49]:
#pass the URL of rapgenius and get returned the song lyrics
def getLyrics (URL):
    page = requests.get(URL)
    html = BeautifulSoup(page.text, "html.parser") # Extract the page's HTML as a string

    # Scrape and return the song lyrics from the HTML
    return html.find("div", class_="lyrics").get_text()

***GETTING NEGATIVE WORDS***

dictionary of words is score_word_dict

In [50]:
#Get requests response "Failed to establish a new connection: [Errno -2]"
url_web = "https://raw.githubusercontent.com/hyharryhuang/AFINN-json/master/AFINN.json"
#We are going to hardcode the Afinn-111 json

afinn_data = rq.get(url_web).json()

In [51]:
score_word_dict = dict(afinn_data) #negative word dictionary

In [52]:
#add firearms
firearm_url = "https://genius.com/Rap-genius-firearms-and-firearm-companies-mentioned-in-rap-lyrics"
scrape_words = getLyrics(firearm_url)
firearm_list = scrape_words[130:].split("\n")

#add firearms in dictionary with sentiment score of -4
for firearm in firearm_list:
    score_word_dict[firearm.lower()] = -4

In [53]:
#add bulleted lists on rapgenius to a list that is returned and adds to score_word_dict
#needs score_word_dict to word, and adds the sentiment of that word (number)
def add_bulleted_lists(URL, number):
    words_drug = getLyrics(URL)
    first_list = words_drug.split("\n")
    new_list = []

    for x in first_list:
        new_list.append(x.replace(u'\u2022 ', ''))

    #add drugs in dictionary with sentiment score of -1
    for thing in new_list:
        score_word_dict[thing.lower()] = number

In [54]:
slang_drugs_url = "https://genius.com/Rap-genius-slang-dictionary-drugs-lyrics"
slang_firearms_url = "https://genius.com/Rap-genius-slang-dictionary-firearms-lyrics"

#adds words into sentiment dictionary
add_bulleted_lists(slang_drugs_url, -2)
add_bulleted_lists(slang_firearms_url, -4)

***REQUEST RAPGENIUS API***

***GET RAPGENIUS LYRICS***

use: get_song_lyrics(artist, song name)

In [122]:
#pass name of artist and song name as uppercase
def get_song_lyrics(artist, song):
    url = "data/lyrics/%s.txt" % artist
    with open(url, "r") as i:
        g = eval(i.read())
    for album in g:
        for name in album["tracks"]:
            if song is name["name"]:
                return name["lyrics"]
            else:
                continue

In [134]:
url = "data/lyrics/%s.txt" % "03 Greedo"
with open(url, "r") as i:
    g = eval(i.read())

g
#print get_song_lyrics("03 Greedo", "Drippin")

{'100 Bands ': None,
 'Bacc to Bacc ': None,
 "Ballin' ": None,
 'Baytoven': 'uh bitch you took too long all my niggas got on i dog a bitch like beethoven ron ron just like my zaytoven ron ron do that shit beat boy thats my metro boomin beat boy taught me i dog a bitch like beethoven ron ron just like my zaytoven beat boy thats my metro boomin we takin all that shit over i dog a bitch like beethoven ron ron just like my zaytoven beat boy thats my metro boomin we takin all that shit over dog a bucket head bitch like a cocker spaniel 03 greedo got it lit just like a roman candle long nose four four call it cheat code next friday feel like joker with the heat though no low-riders but im hoppin in traffic my bitch keep askin ill shoot if its blastin yeah i know that pussy but can tell she a catfish fried foods grillin bitches take your wife too i feel like the nigga who was poppin back in high school twelfth grade sold drugs principal face dog a lil bitch ass like a pomeranian im in dubai 

In [82]:
#**supports get_song_lyrics**
#cleans up lyrics once it gets sent back from rapgenius
#pass parameter as string lyrics
#returns as string of lyrics separated by spaces
def process_lyrics (lyrics):
    #removes stuff in brackets
    new_lyrics = re.sub("\[(.+?)\]", '', lyrics)
    
    #removes special characters and newline
    no_char = ''.join(e for e in new_lyrics if e.isalnum() or e == " " or e == "\n")
    no_char = " ".join(no_char.split("\n"))
    
    #returns string in lowercase
    return no_char.lower()

***ANALYZE SENTIMENT SCORE FOR LYRICS***

In [85]:
#input: artist as string and song name as string
#output: integer sentiment score for song
#necessary: needs get_song_lyrics and process_lyrics as auxillary methods
def analyze_senti(artist, song):
    print get_song_lyrics(artist, song)
    lyrics_list = get_song_lyrics(artist, song).split(" ")
    
    value = 0
    senti = 0
    for word in lyrics_list:
        if word in score_word_dict:
            value = value + int(score_word_dict[word])
        else:
            continue
        senti = value / (1.0 * len(lyrics_list))
    return senti

In [86]:
#input: artist as string for artist name, list of songs in album as list
#output: dict of album with senti scores
def process_album(artist, album):
    total = 0
    album_dict = {}
    for song in album:
        album_dict[song] = analyze_senti(artist, song)
    return album_dict

In [87]:
#input: artist as string and discography where album is key and album songs are list as value
#output: dict of artist discography where key is album and value is dictionary with songs as keys and senti scores as values 
def process_artist(artist, discography):
    total = 0
    artist_dict = {}
    for key, value in discography.iteritems():
        artist_dict[key] = process_album(artist, value)
    return artist_dict

In [88]:
#input: artist as string
#output: discography as dictionary where album are keys and songs are list of values
def get_disc(artist):
    url = "data/albums/%s.txt" % artist.decode("utf-8")
    with open(url, 'r') as f:
        x = eval(f.read())
    discography = {}
    for album in x: #x[0]
        discography[album["name"]] = [re.sub("\([^)]*\)", "", track['track_title']) for track in album['tracks']]
    return discography

In [89]:
def get_rappers():
    
    def validate(name):
        # checks if the name has a
        illegal_chars = "|?:^*\""

        for c in illegal_chars:
            if c == '|':
                if name.find(c) > 0:
                    name = name.split("|")[1]
            if c == "\"":
                if name.find(c) > 0:
                    name = name.replace(r'"','',2)
            else: 
                name = name.replace(c,'')
        return name
    
    with open("data/rappers.txt", "r") as f:
        rappers = f.read()
        
    return [validate(rapper).strip() for rapper in rappers.split("\n")]
rappers = get_rappers() 

In [124]:
x = get_rappers()

for artist in x:
    print artist
    try:
        discography = get_disc(artist)
        print discography
        senti_dict = process_artist(artist, discography)
        artist_url = "data/senti_data/%s.txt" % artist.decode("utf-8")
        file = open(artist_url, "w")
        file.write(str(senti_dict))
        file.close()
        print "done: " + artist
    except AttributeError, SyntaxError:
        continue

03 Greedo
{'The Wolf of Grape Street': ["Drippin'", "Ballin' ", "If I Wasn't Rappin'", 'Bacc to Bacc ', 'Baytoven', 'Beat That Thang Down ', 'Run For Yo Life', 'Look At Me Now', 'Voodoo', '100 Bands ', 'Paranoid Pt. 03 ', 'Vulture ', 'Chase ', 'White Owl ', 'Safety ', 'For My Dawgs', 'Pop It', 'Substance', 'Rude', 'Zoning', 'Never Bend'], "Purple Summer 2: Son Don't Shine": ['Mobbin ', 'Creepin', 'Hands Dirty', 'Everywhere', 'Paranoid', 'Stick Em Up', 'Daylight Savings', 'Told Ya ', 'Fabo', 'Condo Money ', 'Chemistry', 'Invites', 'Bend It Over', 'How Bout Now', 'Neva Make Me Mad', 'Afraid Interlude', 'Yeezy All Winter', 'Paper Plates', 'Blue Hunnid', 'Get Your Life Up', 'Came to Spend Some Money', 'Ransom', 'Bartender', 'Fishscale ', 'Wasted', 'Ilene', 'Baby Sittin', 'Life of the Party ', "Pay Me in Penny's", 'No Label', 'Wake Me Up', 'Undercover Freak ', 'Desperado '], 'God Level': ['Blower ', 'Finally', 'Fall Off', 'Basehead', 'Prayer For My Lost', 'Buckhead ', 'In My Feelings', 'Pra

TypeError: string indices must be integers, not str

In [ ]:
def save_lyrics(artist, discography):
    total = 0
    artist_dict = {}
    for key, value in discography.iteritems():
        artist_dict[key] = process_album(artist, value)
    return artist_dict

In [91]:
def process_album(artist, album):
    total = 0
    album_dict = {}
    for song in album:
        album_dict[song] = get_song_lyrics(artist, song)
    url = "data/lyrics/%s.txt" % artist
    file = open(url, 'w')
    file.write(str(album_dict))
    file.close()

In [ ]:
#input rapper name
#output overall aggression score
def ov_aggression(artist):
    art_dict = process_artist(artist)
    total = 0
    for value in art_dict.iteritems():
        sum_of_all_album = sum(value.values())
        total = total + (sum_of_all_album / (1.0 * len(value)))
    return total / (1.0 * len(art_dict))